In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
from smount_predictors import SeamountTransformer, SeamountCVSplitter
from smount_predictors import SeamountHelp
import plotly.express as px
import pandas as pd
import numpy as np

In [2]:
coords = SeamountHelp.readKMLbounds(Path('data') / 'Seamount_training_zone.kml')
training_data = SeamountHelp.readAndFilterGRD(Path('data') / 'vgg_swot.grd', coords[:2], coords[2:])

In [3]:
tran_inst = SeamountTransformer()
tran_inst.sig = 0.5
tran_inst.TEST_CONV_KERN = np.array([
    [0, 0 ,0],
    [0, 1, 0],
    [0, 0, 0]
    ])

In [4]:
numpy_train = training_data.to_dataframe().reset_index()[['lon', 'lat', 'z']].to_numpy()

In [5]:
trans = tran_inst.transform(numpy_train)

In [6]:
trans

array([[-1.72006415, -1.72006415, -0.08362165],
       [-1.72006415, -1.69600731, -0.07581436],
       [-1.72006415, -1.67195047, -0.07667128],
       ...,
       [ 1.72006415,  1.67195047, -0.08377811],
       [ 1.72006415,  1.69600731, -0.08551055],
       [ 1.72006415,  1.72006415, -0.07337838]])

In [7]:
trans_df = pd.DataFrame(trans, columns=['lon', 'lat', 'z'])
fig = px.scatter(trans_df, x='lon', y='lat', color='z', title='Transformed data')
fig.update_layout(
    autosize=False,
    width=800,
    height=800,
)

In [8]:
from scipy.ndimage import gaussian_filter, convolve
rex = SeamountHelp.xar_from_numpy(trans)
rex['z'].values = convolve(rex['z'].values, SeamountTransformer.TEST_CONV_KERN * 10, mode='constant')

In [9]:
fig = SeamountHelp.plot_xarr(rex.where(rex['z'] > 0.5), name='convolved')
# fig.update_coloraxes(colorscale='PiYG')
fig.show()

In [10]:
current_kern = np.array([
        [0.02370913, 0.06444810, 0.06438915, 0.02370913, 0.06444810],
        [0.06444810, 0.02370913, 0.02370913, 0.06444810, 0.02370913],
        [0.02370913, 0.02370913, 0.00000000, 0.02370913, 0.02370913],
        [0.02370913, 0.06444810, 0.06438915, 0.02370913, 0.06444810],
        [0.06444810, 0.02370913, 0.02370913, 0.06444810, 0.02370913]
        ])
manip = pd.DataFrame(current_kern)
manip[[1, 0, 2, 3, 4]]
manip.to_numpy()

array([[0.02370913, 0.0644481 , 0.06438915, 0.02370913, 0.0644481 ],
       [0.0644481 , 0.02370913, 0.02370913, 0.0644481 , 0.02370913],
       [0.02370913, 0.02370913, 0.        , 0.02370913, 0.02370913],
       [0.02370913, 0.0644481 , 0.06438915, 0.02370913, 0.0644481 ],
       [0.0644481 , 0.02370913, 0.02370913, 0.0644481 , 0.02370913]])

In [11]:
np.repeat(1/15, 16).reshape(4, 4)

array([[0.06666667, 0.06666667, 0.06666667, 0.06666667],
       [0.06666667, 0.06666667, 0.06666667, 0.06666667],
       [0.06666667, 0.06666667, 0.06666667, 0.06666667],
       [0.06666667, 0.06666667, 0.06666667, 0.06666667]])

In [ ]:
points = SeamountHelp.readKMLbounds(Path('data/seamount_training_zone.kml'))
data = SeamountHelp.readAndFilterGRD(Path('data') / 'vgg_swot.grd', points[:2], points[2:])
y = SeamountHelp.readAndFilterGRD(Path('data') / 'vgg_seamounts_labled.nc')
X = data.to_dataframe().reset_index().merge(y.to_dataframe().reset_index(), on=['lat', 'lon'], how='left')
X['z'] = X['z_x']
y = X['Labels']
X = X[['lat', 'lon', 'z']]
index_gen = SeamountCVSplitter(5).split(X, y)
train_index, test_index = next(index_gen)
X_train, X_test = X.iloc[train_index], X.iloc[test_index]
y_train, y_test = y.iloc[train_index], y.iloc[test_index]